In [1]:
pip install tensorflow keras opencv-python numpy matplotlib pandas scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory

train_dir = "C:/MENATL_HEALTH/train"
test_dir = "C:/MENATL_HEALTH/test"

# Load Train & Test dataset
batch_size = 32
img_size = (48, 48)

train_ds = image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale"  # Convert images to grayscale
)

test_ds = image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale"
)

# Class names (emotions)
class_names = train_ds.class_names
print("Emotion Labels:", class_names)



Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.
Emotion Labels: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(48,48,1)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),  # Prevent overfitting
    Dense(len(class_names), activation="softmax")  # Output layer
])

# Compile Model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train Model
history = model.fit(train_ds, validation_data=test_ds, epochs=20)


Epoch 1/20


c:\Users\choud\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


898/898 ━━━━━━━━━━━━━━━━━━━━ 33s 35ms/step - accuracy: 0.2350 - loss: 2.7197 - val_accuracy: 0.3518 - val_loss: 1.6297
Epoch 2/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 28s 31ms/step - accuracy: 0.3400 - loss: 1.6646 - val_accuracy: 0.3902 - val_loss: 1.5574
Epoch 3/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 28s 31ms/step - accuracy: 0.3871 - loss: 1.5737 - val_accuracy: 0.4386 - val_loss: 1.4480
Epoch 4/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 31s 34ms/step - accuracy: 0.4118 - loss: 1.5061 - val_accuracy: 0.4320 - val_loss: 1.4546
Epoch 5/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 703s 784ms/step - accuracy: 0.4388 - loss: 1.4524 - val_accuracy: 0.4663 - val_loss: 1.3850
Epoch 6/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 31s 34ms/step - accuracy: 0.4609 - loss: 1.4090 - val_accuracy: 0.4791 - val_loss: 1.3679
Epoch 7/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 29s 33ms/step - accuracy: 0.4611 - loss: 1.3875 - val_accuracy: 0.4806 - val_loss: 1.3525
Epoch 8/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - accuracy: 0.4793 - loss: 1.3459 - val_accur

In [9]:
model.save("emotion_model.keras")
print("Model saved successfully!")


Model saved successfully!


In [6]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.4837 - loss: 1.4264
Test Accuracy: 48.27%


In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("emotion_model.h5")

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open Webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30,30))

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]  # Extract face
        face = cv2.resize(face, (48,48))  # Resize to match model input
        face = np.expand_dims(face, axis=0)  # Expand dims for batch
        face = np.expand_dims(face, axis=-1) / 255.0  # Normalize

        # Predict Emotion
        prediction = model.predict(face)
        emotion_label = class_names[np.argmax(prediction)]

        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("emotion_model.h5")

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open Webcam
cap = cv2.VideoCapture(0)

# Run for 10 frames (adjustable)
for _ in range(10):
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30,30))

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]  # Extract face
        face = cv2.resize(face, (48,48))  # Resize to match model input
        face = np.expand_dims(face, axis=0)  # Expand dims for batch
        face = np.expand_dims(face, axis=-1) / 255.0  # Normalize

        # Predict Emotion
        prediction = model.predict(face)
        emotion_label = class_names[np.argmax(prediction)]

        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    # Convert frame to RGB (for Matplotlib)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Display the image in Jupyter Notebook
    plt.imshow(frame_rgb)
    plt.axis("off")
    plt.show()

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
